#### Initial setup

In [1]:
pl = (require "pl.import_into")() -- Load Penlight module

torch.setdefaulttensortype("torch.FloatTensor")
torch.manualSeed(1)
math.randomseed(1)

#### Download GTSRB dataset 

In [2]:
 -- Remote GTSRB paths
remote_train_dataset = "http://benchmark.ini.rub.de/Dataset/GTSRB_Final_Training_Images.zip"
remote_test_dataset = "http://benchmark.ini.rub.de/Dataset/GTSRB_Final_Test_Images.zip"
remote_test_dataset_gt = "http://benchmark.ini.rub.de/Dataset/GTSRB_Final_Test_GT.zip"

-- Local path where the dataset will be downloaded
local_dataset_path = "dataset"
if not pl.path.isdir(pl.path.join(local_dataset_path, "GTSRB")) then
    -- Save zip names in local variables
    local train_zip_path = pl.path.join(local_dataset_path, paths.basename(remote_train_dataset))
    local test_zip_path = pl.path.join(local_dataset_path, paths.basename(remote_test_dataset))
    local test_gt_zip_path = pl.path.join(local_dataset_path, paths.basename(remote_test_dataset_gt))
    -- Download dataset
    print("Downloading dataset...")
    os.execute("wget " .. remote_train_dataset .. " -P " .. local_dataset_path .. "; " .. 
                "unzip " .. train_zip_path .. " -d " .. local_dataset_path .. "; " ..
                "rm " .. train_zip_path)
    os.execute("wget " .. remote_test_dataset .. " -P " .. local_dataset_path .. "; " .. 
                "unzip " .. test_zip_path .. " -d " .. local_dataset_path .. "; " ..
                "rm " .. test_zip_path)
    os.execute("wget " .. remote_test_dataset_gt .. " -P " .. local_dataset_path .. "; " ..  
                "unzip " .. test_gt_zip_path .. " -d " .. pl.path.join(local_dataset_path, "GTSRB", "Final_Test") .. "; " ..
                "rm " .. test_gt_zip_path)
    print("Dataset downloaded")
else
    print("Dataset already downloaded")
end
local_dataset_path = pl.path.join(local_dataset_path, "GTSRB")

Dataset already downloaded	


#### Generate dataset database
Generate a dataset database as torch Tensor from a directory of images.

##### Training database

In [3]:
-- Define where training database will be stored at local disk
training_dataset_db = pl.path.join(local_dataset_path, "training_dataset.db")

if not pl.path.isfile(training_dataset_db) then
    -- Generate database of training images
    print("Generating database of training images...")
    training_dataset = {}
    training_dataset.data = {}
    training_dataset.name = {}
    training_dataset.label = {}

    training_images_dirs = pl.dir.getdirectories(pl.path.join(local_dataset_path, "Final_Training", "Images"))
    table.sort(training_images_dirs)

    -- Iterate over classes directories
    for image_dir_idx, image_dir in ipairs(training_images_dirs) do
        local csv_file_name = "GT-" .. pl.path.basename(image_dir) .. ".csv"
        local csv_file_path = pl.path.join(image_dir,csv_file_name)

        local csv_data = pl.data.read(csv_file_path)

        -- Fieldnames returns csv header
        local filename_index = csv_data.fieldnames:index("Filename")
        local class_id_index = csv_data.fieldnames:index("ClassId")

        -- Iterate over images metadata of the traffic sign class being processed (csv rows)
        for image_idx, image_metadata in ipairs(csv_data) do
            local image_name = image_metadata[filename_index]
            local image_path = pl.path.join(image_dir, image_name)
            local image_data = image.load(image_path)
            local image_label = image_metadata[class_id_index] + 1

            -- Rescale image to 48x48
            image_data = image.scale(image_data, 48, 48)

            -- Save info in Lua tables
            table.insert(training_dataset.data, image_data)
            table.insert(training_dataset.name, image_name)
            table.insert(training_dataset.label, image_label)
        end
    end

    -- Convert data and label Lua tables to Tensors.
    -- For image data, create a Tensor of size [number_elements, 3 (RGB channels), 48 (height), 48 (width)].
    -- Generally, torch networks work with BCHW Tensor layouts 
    -- where B is the batch size, C the image channels, H the height and W the width.
    local num_elements = #training_dataset.data
    local image_sample = training_dataset.data[1]
    print("Printing image sample size...")
    print(image_sample:size())
    print("Displaying training image sample...")
    itorch.image(image_sample)
    local channels = image_sample:size(1)
    local height = image_sample:size(2)
    local width = image_sample:size(3)
    local training_images_tensor = torch.Tensor(num_elements, channels, height, width)
    for idx,image_data in ipairs(training_dataset.data) do
        training_images_tensor[idx]:copy(image_data)
    end
    training_dataset.data = training_images_tensor
    print("Printing training dataset data size...")
    print(training_dataset.data:size())

    -- For image labels, convert the table directly to a Tensor
    training_dataset.label = torch.Tensor(training_dataset.label)
    print("Printing training dataset label size...")
    print(training_dataset.label:size())

    -- Save training dataset to local disk
    print("Saving training dataset to ".. training_dataset_db .. " database file ...")
    torch.save(training_dataset_db, training_dataset)
    training_dataset = nil
    training_images_tensor = nil
    print("Training database saved")
else
     print("Training database already exists")   
end

Training database already exists	


##### Testing database

In [4]:
-- Define where testing database will be stored at local disk
testing_dataset_db = pl.path.join(local_dataset_path, "testing_dataset.db")

if not pl.path.isfile(testing_dataset_db) then
    -- Generate database of testing images
    print("Generating database of testing images...")

    testing_dataset = {}
    testing_dataset.data = {}
    testing_dataset.name = {}
    testing_dataset.label = {}

    image_dir = pl.path.join(local_dataset_path, "Final_Test", "Images")
    csv_file_name = "GT-final_test.csv"
    csv_file_path = pl.path.join(local_dataset_path, "Final_Test", csv_file_name)

    csv_data = pl.data.read(csv_file_path)

    -- Fieldnames returns csv header
    local filename_index = csv_data.fieldnames:index("Filename")
    local class_id_index = csv_data.fieldnames:index("ClassId")

    -- Iterate over testing images
    for image_idx, image_metadata in ipairs(csv_data) do
        local image_name = image_metadata[filename_index]
        local image_path = pl.path.join(image_dir, image_name)
        local image_data = image.load(image_path)
        local image_label = image_metadata[class_id_index] + 1

        -- Rescale image to 48x48
        image_data = image.scale(image_data, 48, 48)

        -- Save info in Lua tables
        table.insert(testing_dataset.data, image_data)
        table.insert(testing_dataset.name, image_name)
        table.insert(testing_dataset.label, image_label)
    end


    -- Convert data and label Lua tables to Tensors.
    -- For image data, create a Tensor of size [number_elements, 3 (RGB channels), 48 (height), 48 (width)].
    -- Generally, torch networks work with BCHW Tensor layouts 
    -- where B is the batch size, C the image channels, H the height and W the width.
    local num_elements = #testing_dataset.data
    local image_sample = testing_dataset.data[1]
    print("Printing image sample size...")
    print(image_sample:size())
    print("Displaying testing image sample...")
    itorch.image(image_sample)
    local channels = image_sample:size(1)
    local height = image_sample:size(2)
    local width = image_sample:size(3)
    local testing_images_tensor = torch.Tensor(num_elements, channels, height, width)
    for idx,image_data in ipairs(testing_dataset.data) do
        testing_images_tensor[idx]:copy(image_data)
    end
    testing_dataset.data = testing_images_tensor
    print("Printing testing dataset data size...")
    print(testing_dataset.data:size())

    -- For image labels, convert the table directly to a Tensor
    testing_dataset.label = torch.Tensor(testing_dataset.label)
    print("Printing testing dataset label size...")
    print(testing_dataset.label:size())

    -- Save training dataset to local disk
    print("Saving testing dataset to ".. testing_dataset_db .. " database file ...")
    torch.save(testing_dataset_db, testing_dataset)
    testing_dataset = nil
    print("Testing database saved")
else
    print("Testing database already exists")
end

Testing database already exists	


### Create the Deep Neural Network model

#### Convolutional layers
A convolution is an integral that expresses the amount of overlap of one function g as it is shifted over another function f. It therefore "blends" one function with another. The neural network package supports convolution, pooling, subsampling and other relevant facilities. 

##### Spatial Convolution
Applies a 2D convolution over an input image composed of several input planes. The input tensor in forward(input) is expected to be a 3D tensor (nInputPlane x height x width).
module = nn.SpatialConvolutionMM(nInputPlane, nOutputPlane, kW, kH, [dW], [dH], [padW], [padH])

*Input image*
<img src="images/sign.jpg" width="48" height="48"/>

*Convolutional: neurons activated*
![Conv1 activations](images/conv1_activations.png)

*Convolutional: weights - learned parameters (19k)*
![Conv1 weights](images/conv1_weights_19k_learned_parameters.png)

#####  Spatial Max Pooling
Applies 2D max-pooling operation in kWxkH regions by step size dWxdH steps. The number of output features is equal to the number of input planes.
If the input image is a 3D tensor nInputPlane x height x width, the output image size will be nOutputPlane x oheight x owidth.
module = nn.SpatialMaxPooling(kW, kH [, dW, dH, padW, padH])

![Max-pooling example](http://cs231n.github.io/assets/cnn/maxpool.jpeg)

**Non-linearity layers**
Transfer functions are normally used to introduce a non-linearity after a parameterized layer like Linear and SpatialConvolution. Non-linearities allows for dividing the problem space into more complex regions than what a simple logistic regressor would permit.

*ReLU*
Applies the rectified linear unit (ReLU) function element-wise to the input Tensor, thus outputting a Tensor of the same dimension. ReLU is defined as: f(x) = max(0, x)
f = nn.ReLU([inplace])

*LogSoftMax*
Applies the LogSoftMax function to an n-dimensional input Tensor.
f = nn.LogSoftMax()

**Simple layers**

*View*
This module creates a new view of the input tensor using the sizes passed to the constructor. The parameter sizes can either be a LongStorage or numbers. The method setNumInputDims() allows to specify the expected number of dimensions of the inputs of the modules.
module = nn.View(sizes)

*Linear*
Applies a linear transformation to the incoming data, i.e. y = Ax + b. The input tensor given in forward(input) must be either a vector (1D tensor) or matrix (2D tensor). If the input is a matrix, then each row is assumed to be an input sample of given batch.
module = nn.Linear(inputDimension, outputDimension, [bias = true])

#### Network initialization

In [5]:
require 'nn' -- Load Neural Networks module for Torch

-- Load Nvidia CUDA modules
if pcall(function() require('cudnn') end) then
    print('Using CuDNN backend')
    require 'cunn'
    backend = cudnn
else
    print('Failed to load cudnn backend (is libcudnn.so in your library path?)')
    if pcall(function() require('cunn') end) then
        print('Falling back to legacy cunn backend')
    else
        print('Failed to load cunn backend (is CUDA installed?)')
        print('Falling back to legacy nn backend')
    end
    backend = nn
end

layers = {}
layers.convolutionLayer = backend.SpatialConvolution
layers.poolingLayer = backend.SpatialMaxPooling
layers.nonLinearityLayer = backend.ReLU
layers.softmaxLayer = backend.LogSoftMax


-- Initialize network parameters
num_channels = 3 -- RGB(3 channels) or grayscale (1 channel)
num_classes = 43 -- Number of different traffic signs to be classified
batch_size = 50 -- Number of samples used in each training/validation iteration
pooling_size = 2 -- Divide the input feature map size by 2 in each max-pooling layer
base_image_size = 48 -- width and height

-- Returns the number of output elements for a table of convolution layers and the new height of the image
function convs_noutput(convs, input_size)
    local output = torch.Tensor(1, num_channels, input_size, input_size)
    if backend == cudnn then
        output = torch.CudaTensor(num_channels, input_size, input_size)
    end
    for _, conv in ipairs(convs) do
        if backend == cudnn then conv:cuda() end
        output = conv:forward(output)
    end
    return output:nElement(), output:size(3)
end

Failed to load cudnn backend (is libcudnn.so in your library path?)	
Failed to load cunn backend (is CUDA installed?)	
Falling back to legacy nn backend	


#### Network architecture definition

In [6]:
-- Define main building blocks of the network
conv1 = layers.convolutionLayer(
    num_channels, -- num input feature maps
    75, -- num output feature maps
    5, -- kernel width of the convolution
    5, -- kernel height of the convolution
    1, -- step of the convolution in the width dimension
    1, -- step of the convolution in the height dimension
    2, -- additional zeros added to the input plane data on both sides of width axis
    2  -- additional zeros added to the input plane data on both sides of height axis
)
relu1 = layers.nonLinearityLayer()
max_pool1 = layers.poolingLayer(
    2, -- kernel width
    2, -- kernel height
    2, -- step in the width dimension
    2  -- step in the height dimension
) 

conv_output_size = convs_noutput({conv1,relu1, max_pool1},base_image_size)

fc1 = nn.Sequential()
fc1:add(nn.View(conv_output_size))
fc1:add(nn.Linear(conv_output_size,200))

classifier = nn.Sequential()
classifier:add(nn.View(200))
classifier:add(nn.Linear(200, num_classes))

network = nn.Sequential()
network:add(conv1)
network:add(relu1)
network:add(max_pool1)
network:add(fc1)
network:add(classifier)
-- Loss function
network:add(layers.softmaxLayer())

print("Printing DNN...")
print(network)

Printing DNN...	
nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> output]
  (1): nn.SpatialConvolution(3 -> 75, 5x5, 1,1, 2,2)
  (2): nn.ReLU
  (3): nn.SpatialMaxPooling(2x2, 2,2)
  (4): nn.Sequential {
    [input -> (1) -> (2) -> output]
    (1): nn.View(43200)
    (2): nn.Linear(43200 -> 200)
  }
  (5): nn.Sequential {
    [input -> (1) -> (2) -> output]
    (1): nn.View(200)
    (2): nn.Linear(200 -> 43)
  }
  (6): nn.LogSoftMax
}
{
  gradInput : FloatTensor - empty
  modules : 
    {
      1 : 
        nn.SpatialConvolution(3 -> 75, 5x5, 1,1, 2,2)
        {
          padW : 2
          nInputPlane : 3
          output : FloatTensor - size: 1x75x48x48
          gradInput : FloatTensor - empty
          _type : torch.FloatTensor
          dH : 1
          dW : 1
          nOutputPlane : 75
          padH : 2
          kH : 5
          finput : FloatTensor - size: 1x75x2304
          weight : FloatTensor - size: 75x3x5x5
          gradWeight : FloatTensor - size: 

     }
      2 : 
        nn.ReLU
        {
          inplace : false
          threshold : 0
          _type : torch.FloatTensor
          output : FloatTensor - size: 1x75x48x48
          gradInput : FloatTensor - empty
          val : 0
        }
      3 : 
        nn.SpatialMaxPooling(2x2, 2,2)
        {
          dH : 2
          dW : 2
          kW : 2
          gradInput : FloatTensor - empty
          iwidth : 48
          indices : FloatTensor - size: 1x75x24x24
          output : FloatTensor - size: 1x75x24x24
          _type : torch.FloatTensor
          padH : 0
          ceil_mode : false
          iheight : 48
          kH : 2
          padW : 0
        }
      4 : 
        nn.Sequential {
          [input -> (1) -> (2) -> output]
          (1): nn.View(43200)
          (2): nn.Linear(43200 -> 200)
        }
        {
          gradInput : FloatTensor - empty
          modules : 
            {
              1 : 
                nn.View(43200)
                {
           

      output : FloatTensor - empty
        }
      5 : 
        nn.Sequential {
          [input -> (1) -> (2) -> output]
          (1): nn.View(200)
          (2): nn.Linear(200 -> 43)
        }
        {
          gradInput : FloatTensor - empty
          modules : 
            {
              1 : 
                nn.View(200)
                {
                  _type : torch.FloatTensor
                  output : FloatTensor - empty
                  gradInput : FloatTensor - empty
                  size : LongStorage - size: 1
                  numElements : 200
                }
              2 : 
                nn.Linear(200 -> 43)
                {
                  gradBias : FloatTensor - size: 43
                  weight : FloatTensor - size: 43x200
                  _type : torch.FloatTensor
                  output : FloatTensor - empty
                  gradInput : FloatTensor - empty
                  bias : FloatTensor - size: 43
                  gradWeight : FloatTens

#### Set training parameters

In [6]:
require 'optim'
-- Load Facebook optim package
paths.dofile('Optim.lua')

-- Initialize optimizer config
local optim_state = {
      learningRate = 0.01,
      momentum = 0,
      learningRateDecay = 0,
      weightDecay = 0,
}
optimizer = nn.Optim(network, optim_state)

-- Initizalize criterion. 
-- Given an input and a target, criterions compute a gradient according to a given loss function.
-- Negative log-likelihood for LogSoftMax (multi-class)
criterion = nn.ClassNLLCriterion()

if backend == cudnn then 
    tensor_type = "torch.CudaTensor"
    network:cuda()
    criterion:cuda()
else
    tensor_type = "torch.FloatTensor"
end

/root/torch/install/share/lua/5.1/torch/init.lua:102: nn.Optim has been already assigned a factory
stack traceback:
	[C]: in function 'newmetatable'
	/root/torch/install/share/lua/5.1/torch/init.lua:102: in function 'class'
	/root/dev/Optim.lua:57: in main chunk
	[C]: in function 'dofile'
	[string "require 'optim'..."]:3: in main chunk
	[C]: in function 'xpcall'
	/root/torch/install/share/lua/5.1/itorch/main.lua:209: in function </root/torch/install/share/lua/5.1/itorch/main.lua:173>
	/root/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	/root/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	/root/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	/root/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/root/torch/install/share/lua/5.1/itorch/main.lua:381: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x00406670: 

##### Training and testing the network

In [10]:
-- Define train and test functions
function train()
    local epoch_error = 0
    
    -- generate random training batches
    local indices = torch.randperm(num_samples_training):long():split(batch_size)
    indices[#indices] = nil -- remove last partial batch

    -- preallocate input and target tensors
    local inputs = torch.zeros(batch_size, num_channels, samples_size, samples_size, tensor_type)
    local targets = torch.zeros(batch_size, tensor_type)

    -- Training process
    for t, ind in ipairs(indices) do
        -- get the minibatch
        inputs:copy(train_dataset.data:index(1, ind))
        targets:copy(train_dataset.label:index(1, ind))

        epoch_error = epoch_error + optimizer:optimize(
            optim.sgd, -- optimization algorithm
            inputs, -- input feature maps tensor
            targets, -- output labels tensor
            criterion -- network criterion
        )
        
        xlua.progress(t * batch_size,num_samples_training)
    end
    
    xlua.progress(num_samples_training, num_samples_training)

    print("Training epoch error: " .. epoch_error)
    local loss = epoch_error/num_samples_training
    print("Training mean error (training set) = " .. loss)
    
    return loss
end

function test()
    local epoch_error = 0
    local correct = 0
    local all = 0

    -- generate indices and split them into batches
    local indices = torch.range(1,num_samples_testing):long()
    indices = indices:split(batch_size)

    -- preallocate input and target tensors
    local inputs = torch.zeros(batch_size, num_channels, samples_size, samples_size,tensor_type)
    local targets = torch.zeros(batch_size, tensor_type)

    for t,ind in ipairs(indices) do
        -- last batch may not be full
        local local_batch_size = ind:size(1)
        -- resize prealocated tensors (should only happen on last batch)
        inputs:resize(local_batch_size, num_channels, samples_size, samples_size)
        targets:resize(local_batch_size)

        inputs:copy(testing_dataset.data:index(1,ind))
        targets:copy(testing_dataset.label:index(1,ind))

        -- test samples
        local scores = network:forward(inputs)
        epoch_error = epoch_error + criterion:forward(scores, targets)

        local _, preds = scores:max(2)

        correct = correct + preds:float():eq(targets:float()):sum()
        all = all + preds:size(1)

        xlua.progress(t * batch_size, num_samples_testing)
      end
    
      xlua.progress(num_samples_testing, num_samples_testing)
      
      local loss = epoch_error/num_samples_testing
      print("Testing mean error (test set) = " .. loss)
      local accuracy = correct / all
      print('accuracy % : ', accuracy * 100)
    return loss, accuracy
end

In [11]:
-- Load the training dataset
train_dataset = torch.load(training_dataset_db)
num_samples_training = train_dataset.data:size(1)
samples_size = train_dataset.data:size(3)

-- Load the testing dataset
testing_dataset = torch.load(testing_dataset_db)
num_samples_testing = testing_dataset.data:size(1)

In [12]:
function full_process(total_epochs, save_model)
    local epoch = 1
    
    local plotting_data = {}
    plotting_data.epoch_id = {}
    plotting_data.train_loss = {}
    plotting_data.test_loss = {}
    plotting_data.test_accuracy = {}

    while epoch <= total_epochs do
        print("Starting epoch " .. epoch)
        train_loss = train()
        test_loss, accuracy = test()

        table.insert(plotting_data.epoch_id, epoch)
        table.insert(plotting_data.train_loss, train_loss)
        table.insert(plotting_data.test_loss, test_loss)
        table.insert(plotting_data.test_accuracy, accuracy)
        
        print("Saving network model to disk...")
        torch.save("network_model.t7", network)

        epoch = epoch + 1
    end
    return plotting_data
end

plotting_data = full_process(5)

Starting epoch 1	


[string "-- Define train and test functions..."]:20: attempt to index global 'optim' (a nil value)
stack traceback:
	[string "-- Define train and test functions..."]:20: in function 'train'
	[string "function full_process(total_epochs, save_mode..."]:12: in function 'full_process'
	[string "function full_process(total_epochs, save_mode..."]:28: in main chunk
	[C]: in function 'xpcall'
	/root/torch/install/share/lua/5.1/itorch/main.lua:209: in function </root/torch/install/share/lua/5.1/itorch/main.lua:173>
	/root/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	/root/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	/root/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	/root/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/root/torch/install/share/lua/5.1/itorch/main.lua:381: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x00406670: 

### Plotting training process

In [33]:
Plot = require 'itorch.Plot'
function plot_training_process()
    plot = Plot():circle(plotting_data.epoch_id, plotting_data.train_loss, 'red', 'train_loss')
    plot:circle(plotting_data.epoch_id, plotting_data.test_loss, 'blue', 'test_loss')
    plot:circle(plotting_data.epoch_id, plotting_data.test_accuracy,'green', 'accuracy')
    plot:title('Training process'):redraw()
    plot:xaxis('epoch'):yaxis('loss/accuracy'):redraw()
    plot:legend('true')
    plot:redraw()
end
plot_training_process()

#### Improving the network
http://people.idsia.ch/~juergen/nn2012traffic.pdf

In [34]:
-- New neural network architecture to improve previous results

conv1 = layers.convolutionLayer(num_channels,100,7,7,1,1,2,2)
relu1 = layers.nonLinearityLayer()
max_pool1 = layers.poolingLayer(2,2,2,2)
c_norm1 = nn.SpatialContrastiveNormalization(100)
conv_module1 = nn.Sequential():add(conv1):add(relu1):add(max_pool1):add(c_norm1)


conv2 = layers.convolutionLayer(100,150,4,4,1,1,2,2)
relu2 = layers.nonLinearityLayer()
max_pool2 = layers.poolingLayer(2,2,2,2)
c_norm2 = nn.SpatialContrastiveNormalization(150)
conv_module2 = nn.Sequential():add(conv2):add(relu2):add(max_pool2):add(c_norm2)

conv3 = layers.convolutionLayer(150,250,4,4,1,1,2,2)
relu3 = layers.nonLinearityLayer()
max_pool3 = layers.poolingLayer(2,2,2,2)
c_norm3 = nn.SpatialContrastiveNormalization(250)
conv_module3 = nn.Sequential():add(conv3):add(relu3):add(max_pool3):add(c_norm3)

conv_output_size = convs_noutput({conv_module1,conv_module2,conv_module3},base_image_size)

fc = nn.Sequential()
fc:add(nn.View(conv_output_size))
fc:add(nn.Linear(conv_output_size,300))
fc:add(layers.nonLinearityLayer())

classifier = nn.Sequential()
classifier:add(nn.View(300))
classifier:add(nn.Linear(300, num_classes))

network = nn.Sequential()
network:add(conv_module1)
network:add(conv_module2)
network:add(conv_module3)
network:add(fc)
network:add(classifier)
-- Loss function deleted

print("Printing DNN...")
print(network)

Printing DNN...	


nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> output]
  (1): nn.Sequential {
    [input -> (1) -> (2) -> (3) -> (4) -> output]
    (1): cudnn.SpatialConvolution(3 -> 100, 7x7, 1,1, 2,2)
    (2): cudnn.ReLU
    (3): cudnn.SpatialMaxPooling(2x2, 2,2)
    (4): nn.SpatialContrastiveNormalization
  }
  (2): nn.Sequential {
    [input -> (1) -> (2) -> (3) -> (4) -> output]
    (1): cudnn.SpatialConvolution(100 -> 150, 4x4, 1,1, 2,2)
    (2): cudnn.ReLU
    (3): cudnn.SpatialMaxPooling(2x2, 2,2)
    (4): nn.SpatialContrastiveNormalization
  }
  (3): nn.Sequential {
    [input -> (1) -> (2) -> (3) -> (4) -> output]
    (1): cudnn.SpatialConvolution(150 -> 250, 4x4, 1,1, 2,2)
    (2): cudnn.ReLU
    (3): cudnn.SpatialMaxPooling(2x2, 2,2)
    (4): nn.SpatialContrastiveNormalization
  }
  (4): nn.Sequential {
    [input -> (1) -> (2) -> (3) -> output]
    (1): nn.View(9000)
    (2): nn.Linear(9000 -> 300)
    (3): cudnn.ReLU
  }
  (5): nn.Sequential {
    [input -> (1) -> (2) -> 

         cudnn.ReLU
                {
                  inplace : false
                  mode : CUDNN_ACTIVATION_RELU
                  _type : torch.CudaTensor
                  output : CudaTensor - size: 100x46x46
                  gradInput : CudaTensor - empty
                  nElem : 211600
                  iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x40fa9da0
                  activDesc : cdata<struct cudnnActivationStruct *[1]>: 0x400d18f8
                }
              3 : 
                cudnn.SpatialMaxPooling(2x2, 2,2)
                {
                  dH : 2
                  dW : 2
                  kW : 2
                  gradInput : CudaTensor - empty
                  oDesc : cdata<struct cudnnTensorStruct *[1]>: 0x41084550
                  iSize : LongStorage - size: 4
                  iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x4049ef30
                  poolDesc : cdata<struct cudnnPoolingStruct *[1]>: 0x40f03c48
                  mode : CUDNN_POOLI

          {
              1 : 
                cudnn.SpatialConvolution(100 -> 150, 4x4, 1,1, 2,2)
                {
                  padW : 2
                  pad : table: 0x41c12c00
                  nInputPlane : 100
                  output : CudaTensor - size: 150x24x24
                  gradInput : CudaTensor - empty
                  input_slice : CudaTensor - size: 1x100x23x23
                  iSize : LongStorage - size: 4
                  convDesc : cdata<struct cudnnConvolutionStruct *[?]>: 0x40faa1b8
                  output_offset : 86400
                  weightDesc : cdata<struct cudnnFilterStruct *[?]>: 0x41d802c8
                  _type : torch.CudaTensor
                  bias : CudaTensor - size: 150
                  padH : 2
                  output_slice : CudaTensor - size: 1x150x24x24
                  stride : table: 0x41c12c40
                  groups : 1
                  gradBias : CudaTensor - size: 150
                  dW : 1
                  nOutputP

                  mode : CUDNN_ACTIVATION_RELU
                  _type : torch.CudaTensor
                  output : CudaTensor - size: 150x24x24
                  gradInput : CudaTensor - empty
                  nElem : 86400
                  iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x404f4560
                  activDesc : cdata<struct cudnnActivationStruct *[1]>: 0x40ca7e90
                }
              3 : 
                cudnn.SpatialMaxPooling(2x2, 2,2)
                {
                  dH : 2
                  dW : 2
                  kW : 2
                  gradInput : CudaTensor - empty
                  oDesc : cdata<struct cudnnTensorStruct *[1]>: 0x40d13248
                  iSize : LongStorage - size: 4
                  iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x402c6fc0
                  poolDesc : cdata<struct cudnnPoolingStruct *[1]>: 0x4063d030
                  mode : CUDNN_POOLING_MAX
                  _type : torch.CudaTensor
                  padH 

          output : CudaTensor - size: 150x12x12
        }
      3 : 
        nn.Sequential {
          [input -> (1) -> (2) -> (3) -> (4) -> output]
          (1): cudnn.SpatialConvolution(150 -> 250, 4x4, 1,1, 2,2)
          (2): cudnn.ReLU
          (3): cudnn.SpatialMaxPooling(2x2, 2,2)
          (4): nn.SpatialContrastiveNormalization
        }
        {
          gradInput : CudaTensor - empty
          modules : 
            {
              1 : 
                cudnn.SpatialConvolution(150 -> 250, 4x4, 1,1, 2,2)
                {
                  padW : 2
                  pad : table: 0x4086fe98
                  nInputPlane : 150
                  output : CudaTensor - size: 250x13x13
                  gradInput : CudaTensor - empty
                  input_slice : CudaTensor - size: 1x150x12x12
                  iSize : LongStorage - size: 4
                  convDesc : cdata<struct cudnnConvolutionStruct *[?]>: 0x410b94c8
                  output_offset : 42250
              

            oSize : LongStorage - size: 4
                  iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x408387f8
                  dH : 1
                  autotunerHash : -dimA1,150,12,12 -filtA250,150,4,4 1,250,13,13 -padA2,2 -convStrideA1,1 CUDNN_DATA_FLOAT
                }
              2 : 
                cudnn.ReLU
                {
                  inplace : false
                  mode : CUDNN_ACTIVATION_RELU
                  _type : torch.CudaTensor
                  output : CudaTensor - size: 250x13x13
                  gradInput : CudaTensor - empty
                  nElem : 42250
                  iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x4004c6f8
                  activDesc : cdata<struct cudnnActivationStruct *[1]>: 0x4123f8d0
                }
              3 : 
                cudnn.SpatialMaxPooling(2x2, 2,2)
                {
                  dH : 2
                  dW : 2
                  kW : 2
                  gradInput : CudaTensor - empty
     

{
              1 : 
                nn.View(9000)
                {
                  _type : torch.FloatTensor
                  output : FloatTensor - empty
                  gradInput : FloatTensor - empty
                  size : LongStorage - size: 1
                  numElements : 9000
                }
              2 : 
                nn.Linear(9000 -> 300)
                {
                  gradBias : FloatTensor - size: 300
                  weight : FloatTensor - size: 300x9000
                  _type : torch.FloatTensor
                  output : FloatTensor - empty
                  gradInput : FloatTensor - empty
                  bias : FloatTensor - size: 300
                  gradWeight : FloatTensor - size: 300x9000
                }
              3 : 
                cudnn.ReLU
                {
                  gradInput : FloatTensor - empty
                  inplace : false
                  _type : torch.FloatTensor
                  output : FloatTensor - em

                  gradInput : FloatTensor - empty
                  bias : FloatTensor - size: 43
                  gradWeight : FloatTensor - size: 43x300
                }
            }
          _type : torch.FloatTensor
          output : FloatTensor - empty
        }
    }
  _type : torch.FloatTensor
  output : FloatTensor - empty
}


In [35]:
-- Initialize optimizer config
local optim_state = {
      learningRate = 0.01,
      momentum = 0,
      learningRateDecay = 0,
      weightDecay = 0,
}
optimizer = nn.Optim(network, optim_state)
-- Set new criterion-> CrossEntropyCriterion: combines LogSoftMax and ClassNLLCriterion
criterion = nn.CrossEntropyCriterion()

if backend == cudnn then
    network:cuda()
    criterion:cuda()
end

In [36]:
-- Load the training dataset
train_dataset = torch.load(training_dataset_db)
num_samples_training = train_dataset.data:size(1)
samples_size = train_dataset.data:size(3)

-- Load the testing dataset
testing_dataset = torch.load(testing_dataset_db)
num_samples_testing = testing_dataset.data:size(1)

In [37]:
function global_normalization(dataset, mean, std)
  local std = std or dataset.data:std()
  local mean = mean or dataset.data:mean()
  dataset.data:add(-mean)
  dataset.data:div(std)
  return mean, std
end

function local_normalization(dataset)
  local norm_kernel = image.gaussian1D(7)
  local norm = nn.SpatialContrastiveNormalization(3,norm_kernel)
  local batch = 200
  local dataset_size = dataset.data:size(1)
  for i=1,dataset_size,batch do
    local local_batch = math.min(dataset_size,i+batch) - i
    local normalized_images = norm:forward(dataset.data:narrow(1,i,local_batch))
    dataset.data:narrow(1,i,local_batch):copy(normalized_images)
  end
end

-- Global normalization
mean, std = global_normalization(train_dataset)
global_normalization(testing_dataset, mean, std)
print("Global normalization done")

-- Local normalization
local_normalization(train_dataset)
local_normalization(testing_dataset)
print("Local normalization done")

Global normalization done	


Local normalization done	


In [ ]:
plotting_data = full_process(1)
--plotting_data = nil
-- if not pl.path.isfile("network_model.t7") then
--     plotting_data = full_process(1)
-- else
--     network = torch.load("network_model.t7")
-- end

Starting epoch 1	


 [================================>.............]                                ETA: 38s237ms | Step: 3ms 

In [ ]:
if plotting_data ~= nil then plot_training_process() end

#### Test 1 sample

In [ ]:
sample_img = image.load("images/sign.jpg")
sample_img = image.scale(sample_img, 48, 48)
print("Sample image...")
itorch.image(sample_img)

sample_img:add(-mean)
sample_img:div(std)
print("Globally normalized sample image...")
itorch.image(sample_img)

norm_kernel = image.gaussian1D(7)
local_normalizer = nn.SpatialContrastiveNormalization(3,norm_kernel)
sample_img:copy(local_normalizer:forward(sample_img))
print("Locally normalized sample image")
itorch.image(sample_img)

In [ ]:
samples_tensor = torch.Tensor(1,sample_img:size(1), sample_img:size(2), sample_img:size(3)):fill(0)
samples_tensor[1]:copy(sample_img)
if backend == cudnn then
    samples_tensor = samples_tensor:cuda()
end

local scores = network:forward(samples_tensor)
print("Scores...")
print(scores)
local _, prediction = scores:max(1)
print("Prediction class: " .. prediction[1] - 1)
-- Check prediction at http://arcos.io/spain_tsc/classes.html